In [1]:
import pdfplumber

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

# Example usage
pdf_path = "resume/Fyzan_Ahammad_Resume.pdf"
resume_text = extract_text_from_pdf(pdf_path)
print(resume_text)

D Fyzan Ahammad
(cid:211) 8309648519  fyzanahammad@gmail.com fl LinkedIn (cid:135) Github (cid:140) Portfolio
Education
Jamia Millia Islamia, New Delhi 2020 – 2024
Bachelor of Technology in Electronics and Communication Engineering GPA - 8.6
Sri Medha, Anantapur 2017 – 2019
Class 12 Percentage - 92%
Experience
Deep Learning and Research Intern | Enord - AI on Edge Drone Tech Start-Up Feb 2024 - Present
• Developed and fine-tuned computer vision models using YOLOx object detection framework, achieving over 90%
accuracy on projects focused on runway foreign object detection and solar panel hotspot detection.
• Annotated and prepared datasets comprising 30,000 images for runway foreign object detection and 5,000 images for
solar panel hotspot detection, facilitating robust model training.
• Trained the YOLOx Tiny Model with 5 million parameters on AWS EC2 instance over a period of 2 days,
employing SSH access for streamlined model optimization and achieving significant performance gains.


In [4]:
import os

import google.generativeai as genai

from dotenv import load_dotenv

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate

d:\anaconda3\envs\rough\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import nest_asyncio
nest_asyncio.apply()

# Load environment variables
load_dotenv()

# Configure GenAI with the API key
genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))

In [8]:
import os
from dotenv import load_dotenv

import google.generativeai as genai

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate

import nest_asyncio
nest_asyncio.apply()

# Load environment variables
load_dotenv()

# Configure GenAI with the API key
genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))

# Function to create the conversational chain
def get_conversational_chain():
    template = """
    you are a chatbot trained to collect information for creating a resume. 
    Please ask for each piece of information one by one and confirm the user input before proceeding to the next question.
    Ask for the following details in order: 
    1. Full name
    2. Email address
    3. Phone number
    4. LinkedIn URL
    5. Education details (e.g., degree, university, year)
    6. Project descriptions
    Context:\n{context}\n
    Question:\n{question}\n
    Answer:
    """
    model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3)
    prompt = PromptTemplate(template=template, input_variables=["context", "question"])
    chain = load_qa_chain(model, prompt=prompt, verbose=True)
    return chain

# Function to get user information
def get_user_info(chain, extracted_text):
    user_info = {}
    prompts = [
        ("name", "Please enter your full name."),
        ("email", "Can you provide your email address?"),
        ("phone", "What is your phone number?"),
        ("linkedin", "Please enter your LinkedIn URL."),
        ("education", "Please provide details of your education (e.g., degree, university, year)."),
        ("projects", "Describe a project you have worked on."),
        # Add more prompts as needed
    ]

    context = extracted_text
    
    for key, prompt in prompts:
        response = chain({"context": context, "question": prompt})
        user_info[key] = response['text'].strip()
        context += f"\n{prompt}\n{response['text']}\n"
    
    return user_info

# Main function to run the application
if __name__ == "__main__":
    chain = get_conversational_chain()
    extracted_text = extract_text_from_pdf("resume/Fyzan_Ahammad_Resume.pdf")
    user_info = get_user_info(chain, extracted_text)
    for key, value in user_info.items():
        print(f"{key.capitalize()}: {value}")




> Entering new StuffDocumentsChain chain...


ValueError: Missing some input keys: {'input_documents'}